# Building a Spam Filter for SMS Messages

## Introduction

In this project, we use the multinomial Naive Bayes algorithm to build a spam filter for SMS messages. The Naive Bayes algorithm is used to teach the computer how to classify new messages as spam or non-spam based on a set of human-classified messages. Using the algorithm and a finite dataset, the computer learns how humans classify messages and then applies this knowledge to estimate probabilities of new messages belonging to one class or another. The goal of this project is to create a spam filter that accurately classifies messages at least 80% of the time.

For our spam filter, we use the multinomial Naive Bayes algorithm and a dataset of 5,572 human-classified SMS messages to teach the computer how to decipher between spam and non-spam. The dataset, put together by Tiago A. Almeida and Jose Maria Gomez Hidalgo, can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) or from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). More details on the dataset, including its collection, usage, and associated publications, can be found [here](https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

## Exploring the Dataset

We begin our project by reading in and exploring the dataset.

In [1]:
import pandas as pd
sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                       names=['Label', 'SMS'])

In [2]:
sms_spam.shape

(5572, 2)

In [3]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam_percent = ((sms_spam['Label'] == 'spam').sum() / 
                sms_spam.shape[0] * 100)
ham_percent = ((sms_spam['Label'] == 'ham').sum() /
               sms_spam.shape[0] * 100)

In [5]:
print(spam_percent, ham_percent)

13.406317300789663 86.59368269921033


The dataset consists of 5,572 rows and 2 columns. Each row corresponds to a distinct SMS message. The first column contains the classification or label of the SMS message, with 'spam' indicating a spam message and 'ham' indicating a non-spam message. The second column contains the SMS message itself. Of the 5,572 human-classified SMS messages, 13.4% are spam, and 86.6% are non-spam.

## Generating Training and Test Sets

Before we can begin building our spam filter software, we need to divide our dataset into two separate pieces: a **training set** and a **test set**. The training set will be used to teach, or "train", the computer to classify new messages, and the test set will be used  to assess the efficiency of the filter, i.e. how well the computer classified new messages.

Splitting the dataset this way is essential in order to avoid bias in evaluating the final filter designed from the training set. With a dedicated test set, we will be able to directly compare the classifications produced by the algorithm to those performed by humans in order to calculate the efficieny of the spam filter.

For this project, we will randomly select 80% of the dataset to be training data and 20% to be test data. This allows us to train on as much data as possible while still maintaining enough data to test with later. We randomize and split the dataset below.

In [6]:
# randomize entire dataset
sms_rndm = sms_spam.sample(frac=1, random_state=1)

In [7]:
# split randomized dataset into training and test sets
index_80p = round(sms_rndm.shape[0] * 0.8)
sms_train = sms_rndm[0:index_80p].reset_index(drop=True)
sms_test = sms_rndm[index_80p:].reset_index(drop=True)

In [8]:
# check sizes of new sets
print(sms_train.shape, sms_test.shape)

(4458, 2) (1114, 2)


In [9]:
# find percentage of spam/ham in each set
train_spam_percent = ((sms_train['Label'] == 'spam').sum() / 
                       sms_train.shape[0] * 100)
train_ham_percent = ((sms_train['Label'] == 'ham').sum() /
                      sms_train.shape[0] * 100)
test_spam_percent = ((sms_test['Label'] == 'spam').sum() / 
                      sms_test.shape[0] * 100)
test_ham_percent = ((sms_test['Label'] == 'ham').sum() /
                     sms_test.shape[0] * 100)
print(train_spam_percent, train_ham_percent)
print(test_spam_percent, test_ham_percent)

13.458950201884253 86.54104979811575
13.195691202872531 86.80430879712748


The new training and test sets contain 4,458 and 1,114 rows, respectively, and approximately the same percentages of spam and non-spam messages each as the original full dataset. The similar percentages in each set indicate they are both representative of the whole dataset. 

## Cleaning the Data

We will train the data to classify messages by calculating the conditional probability of a message belonging to a certain class given the words it contains using Bayes's Theorem: 
$$P(\mathrm{class} ~|~ w_1,...,w_n) = P(\mathrm{class}) \times \prod_{i=1}^n P(w_i ~|~ \mathrm{class}).$$

In order to perform the necessary probability calculations, we first need to clean and reformat the training data such that we can more easily extract the information needed. Below, we transform the `SMS` column into a series of columns representing the entire vocabulary contained in all of the SMS messages, with each column corresponding to a single unique word in the vocabulary. The value of the entry in each of these new columns then represents the count of the corresponding word in the individual message of the given row. Capitalization and punctuation are ignored in the vocabulary.

We begin the data cleaning by removing all punctuation from the SMS messages and transforming the messages to all lowercase.

In [10]:
# remove punctuation and transform to lowercase
import re
sms_train['SMS'] = sms_train['SMS'].str.replace('\W', ' ')
sms_train['SMS'] = sms_train['SMS'].str.lower()

In [11]:
sms_train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Next, we build a vocabulary list of all of the unique words in the SMS messages.

In [12]:
# build vocabulary list for messages in training set
sms_train['SMS'] = sms_train['SMS'].str.split()
vocabulary = []
for sms in sms_train['SMS']:
    for word in sms:
        vocabulary.append(str(word))

In [13]:
# transform to list of unique words
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

We then create a dictionary that maps each unique word in the vocabulary to a list of the number of times it appears in each SMS message.

In [14]:
word_counts_per_sms = {vocab : [0] * len(sms_train['SMS']) 
                       for vocab in vocabulary}
for index, sms in enumerate(sms_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

Finally, we can use the dictionary to transform the dataset into the desired format, with each message represented by a series of word counts from the vocabulary.

In [15]:
sms_train_word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

In [16]:
sms_train_reform = pd.concat([sms_train['Label'], 
                              sms_train_word_counts_per_sms], 
                             axis=1)

In [17]:
sms_train_reform.head()

,Label,0,00,000,000pes,008704050406,0089,01223585334,02,0207,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Probabilities

Now that the training dataset is in the proper format, we are ready to begin calculating the probability equations needed for the Naive Bayes algorithm of our spam filter. The algorithm classifies new messages by comparing the conditional probabilities of a new message being either spam or non-spam given the words the message contains. In other words, we need to calculate the following probability equations:

$$P(\mathrm{Spam/Ham ~|~ w_1,...,w_n}) \propto P(\mathrm{Spam/Ham}) \cdot \prod_{i=1}^n P(w_i ~|~ \mathrm{Spam/Ham}),$$

where 

$$P(\mathrm{Spam/Ham}) = \frac{\mathrm{number~of~Spam/Ham~classified~training~set~messages}}{\mathrm{number~of~training~set~messages}},$$

and

$$P(w_i ~|~ \mathrm{Spam/Ham}) = \frac{N_{w_i ~|~ \mathrm{Spam/Ham}} + \alpha}{N_\mathrm{Spam/Ham} + \alpha \cdot N_\mathrm{Vocabulary}}.$$

$N_\mathrm{Spam/Ham}$ is the total number of words in all messages of a given classification (either spam or non-spam), $N_{w_i ~|~ \mathrm{Spam/Ham}}$ is the number of times the given word occurs in all messages of the specified classification, $N_\mathrm{vocabulary}$ is the number of unique words in the vocabulary, and $\alpha$ is a smoothing parameter. For our filter, we use Lapalace smoothing, where $\alpha = 1$.

### Calculating Constants

We begin by calculating the probability equations' constants, the values of which remain fixed, assuming no changes are made to the training set, across all new messages, regardless of their contents. The following are constants of the probability equations:
- $P(\mathrm{Spam})$: the probability of a message being spam, or the fraction of messages in the training set that are classified as spam;
- $P(\mathrm{Ham})$: the probability of a message being non-spam, or the fraction of messages in the training set that are classified as non-spam;
- $N_\mathrm{Spam}$: the total number of words in all spam messages;
- $N_\mathrm{Ham}$: the total number of words in all non-spam messages;
- $N_\mathrm{Vocabulary}$: the number of unique words in the vocabulary;
- $\alpha = 1$: the Laplace smoothing parameter.

We calculate and print their values below.

In [18]:
p_spam = ((sms_train_reform['Label'] == 'spam').sum() /
          sms_train_reform.shape[0])
p_ham = ((sms_train_reform['Label'] == 'ham').sum() /
         sms_train_reform.shape[0])

In [19]:
sms_train_reform['word_ct'] = sms_train_reform.iloc[:,1:].sum(axis=1)

In [20]:
n_spam = (sms_train_reform
          [sms_train_reform['Label'] == 'spam']['word_ct'].sum())
n_ham = (sms_train_reform
         [sms_train_reform['Label'] == 'ham']['word_ct'].sum())

In [21]:
n_vocab = len(vocabulary)
alpha = 1

In [22]:
print(p_spam, p_ham, n_spam, n_ham, n_vocab, alpha)

0.13458950201884254 0.8654104979811574 15190 57237 7783 1


### Calculating Parameters

Next, we calculate the parameters $P(w_i ~|~ \mathrm{Spam/Ham})$, or the probabilities for each word in our vocabulary. These parameters values also remain fixed for all new messages as long as our training set does not change. This means we can calculate the conditional probabilities given the message is spam or non-spam for all the words in our vocabulary of our training set. From these pre-calculated parameters and constants, our filter will then be able to very quickly classify a new message as spam or not.

Below, we calculate the parameters 
$$P(w_i ~|~ \mathrm{Spam/Ham}) = \frac{N_{w_i ~|~ \mathrm{Spam/Ham}} + \alpha}{N_\mathrm{Spam/Ham} + \alpha \cdot N_\mathrm{Vocabulary}}$$

for each word in the vocabulary.

In [23]:
# initialize probability dictionaries
spam_dict = {vocab : 0 for vocab in vocabulary}
ham_dict = {vocab : 0 for vocab in vocabulary}

# split training set into spam and non-spam messages
train_spam = sms_train_reform[sms_train_reform['Label'] == 'spam']
train_ham = sms_train_reform[sms_train_reform['Label'] == 'ham']

## sum number of times each word occurs in each message type
#train_spam_sum = train_spam.sum()
#train_ham_sum = train_ham.sum()

In [25]:
# loop over unique words in vocabulary
for word in vocabulary:
    # count number of times word appears in messages
    n_word_spam = train_spam[word].sum()
    n_word_ham = train_ham[word].sum()
    
    # calculate parameters
    p_word_spam = (n_word_spam + alpha) / (n_spam + alpha * n_vocab)
    p_word_ham = (n_word_ham + alpha) / (n_ham + alpha * n_vocab)
    
    # add parameters to dictionaries
    spam_dict[word] = p_word_spam
    ham_dict[word] = p_word_ham

## Building the Spam Filter

With the necessary probability constants and parameters in hand, we are ready to begin building the spam filter. We define the spam filter as a function that takes a new message as input, calculates the probabilities that the message is spam and non-spam given the words contained, compares those probabilities, and outputs a classification for the given message based on which probability is larger. The probabilities the spam filter calculates in the function are
$$P(\mathrm{Spam/Ham ~|~ w_1,...,w_n}) \propto P(\mathrm{Spam/Ham}) \cdot \prod_{i=1}^n P(w_i ~|~ \mathrm{Spam/Ham}),$$

where the probabilities $P(\mathrm{Spam/Ham})$ and $P(w_i ~|~ \mathrm{Spam/Ham})$ have already been calculated above.

In [26]:
def classify(message):
    """
    Classify message as spam or ham (non-spam).
    
    Parameters
    ----------
    message : str
        Message to classify.
    """
    
    # clean msg: remove punctuation, make lowercase, split into list
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # calculate p_spam/ham_given_message
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    # --> multiply probs by probs for given word in dict
    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]
        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham"message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities; have a human classify this!')

In [27]:
msg1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
msg2 = "Sounds good, Tom, then see u there"
classify(msg1)
classify(msg2)

P(Spam|message): 1.3481290211300841e-25
P(Ham"message): 1.9368049028589875e-27
Label: Spam
P(Spam|message): 2.4372375665888117e-25
P(Ham"message): 3.687530435009238e-21
Label: Ham


## Testing the Filter

With our spam filter now in place, we want to test its capabiities on our test set of data. Below, we modify our filtering function to return, rather than print, a classification for each input message. We then run the filter over all the messages in our test set, the results of which we then compare to the actual human-classified label associated with each message in order to calculate the accuracy of our filter. 

The accuracy, defined as the fraction of messages that are correctly classified, is a metric of how well the predicted values compare to the actual values in our dataset, or of how well our spam filter performs in terms of classifying new messages as spam or non-spam.

In [30]:
def classify_test_set(message):
    """
    Classify message as spam or ham (non-spam).
    
    Parameters
    ----------
    message : str
        Message to classify.
        
    Returns
    -------
    str
        Classification of message as spam or non-spam.
    """
    
    # clean msg: remove punctuation, make lowercase, split into list
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # calculate p_spam/ham_given_message
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    # --> multiply probs by probs for given word in dict
    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]
        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [32]:
# predict classifications for test set
sms_test['predicted'] = sms_test['SMS'].apply(classify_test_set)
sms_test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [38]:
# measure accuracy of spam filter
correct = 0
total = sms_test.shape[0]
for row in sms_test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
accuracy = correct / total
print(accuracy)

0.9874326750448833


The accuracy for our Naive Bayes spam filter is 98.74%, which is excellent!

## Conclusion

In this project, we built a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our filter had an accuracy of nearly 99% for our test set.